In [1]:
# Setup
import geopandas as gpd
import pandas as pd
import dbfread as dbf

# Reading the shapefile as a GeoDataFrame
wa_blocks = gpd.read_file("shapefile/saep_bg10.shp")

In [2]:
# Step 1: Determining urban/rural classification based on population density

# Creating new columns with dummy values
wa_blocks['density'] = None
wa_blocks['urb_or_rur'] = None

for index, row in wa_blocks.iterrows():
    if row['ALANDMI'] == 0: # Some values for the land areas are 0??
        wa_blocks.loc[index, 'density'] = 0 # Avoids dividing by zero problem
    else:
        wa_blocks.loc[index, 'density'] = (row['POP2018'] / row['ALANDMI']) # calculate the density field
            
for index, row in wa_blocks.iterrows():
    if row['density'] >= 1000: # Population density is greater than or equal to 1000 people per square mile
        wa_blocks.loc[index, 'urb_or_rur'] = 'Urban'
    else: # Density is fewer than 1000 per square mile
        wa_blocks.loc[index, 'urb_or_rur'] = 'Rural'

In [3]:
# Delete this later!! - just for checking that the calculated columns look okay
# .loc does position-based selection
# first part is the row, second part is the column(s)

print wa_blocks.loc[10:19,['POP2018', 'ALANDMI', 'density', 'urb_or_rur']]

     POP2018  ALANDMI  density urb_or_rur
10  1356.963    2.519  538.691      Rural
11  1308.087    0.323   4049.8      Urban
12  1824.879    0.988  1847.04      Urban
13  2409.738    3.452   698.07      Rural
14  2968.659    1.330  2232.07      Urban
15  1447.847  185.542  7.80334      Rural
16   821.095  320.618  2.56098      Rural
17  2280.965   11.314  201.606      Rural
18  1123.780    1.449  775.556      Rural
19  1095.111    0.387  2829.74      Urban


In [33]:
# Steps 2 and 3: Calculating percentage of the pop/area is urbanized

# New variables for the urban stuff
totalUrbPop = 0
totalUrbArea = 0

# Variables for total figures
totalPop = wa_blocks.POP2018.sum()
totalAr = wa_blocks.ALANDMI.sum()

for index, row in wa_blocks.iterrows():
    if row['urb_or_rur'] == 'Urban':
        totalUrbPop = totalUrbPop + wa_blocks.loc[index, 'POP2018']
        totalUrbArea = totalUrbArea + wa_blocks.loc[index, 'ALANDMI']

print "Estimation for the percentage of the poulation in Washington that is urbanized is " + str(round(totalUrbPop / totalPop * 100, 2)) + "% of people."
print "Estimation for the percentage of urbanized land area in Washington is " + str(round(totalUrbArea / totalAr * 100, 2)) + "%."

Estimation for the percentage of the poulation in Washington that is urbanized is 73.06% of people.
Estimation for the percentage of urbanized land area in Washington is 2.28%.


In [5]:
# Delete later - just for looking at calculatd columns
#wa_blocks.groupby('urb_or_rur').sum().POP2018
#wa_blocks.loc[wa_blocks['urb_or_rur'] == 'Rural', 'POP2018'].sum()
print(wa_blocks.loc[[1250], ['POP2018', 'ALANDMI', 'density', 'urb_or_rur']])

       POP2018  ALANDMI density urb_or_rur
1250  1829.191    0.033   55430      Urban


In [6]:
# Step 4 setup - looking at previous decade
# Need to calculate 2008 density and categories first

# Creating new columns with dummy values
wa_blocks['2008density'] = None
wa_blocks['2008urb_or_rur'] = None
wa_blocks['block_change'] = None

for index, row in wa_blocks.iterrows():
    if row['ALANDMI'] == 0: # Land area field is 0
        wa_blocks.loc[index, '2008density'] = 0 # Avoids dividing by zero problem
        wa_blocks.loc[index, '2008urb_or_rur'] = 'Rural' # Automatically classified as rural
    else:
        wa_blocks.loc[index, '2008density'] = (row['POP2008'] / row['ALANDMI']) # Calculates density from population and land area
        if wa_blocks.loc[index, '2008density'] >= 1000: # Density greater than 1000
            wa_blocks.loc[index, '2008urb_or_rur'] = 'Urban'
        else: # Density less than 1000 per square mile
            wa_blocks.loc[index, '2008urb_or_rur'] = 'Rural'

In [7]:
# Step 4 getting the categorical change

for index, row in wa_blocks.iterrows():
    c2018 = row['urb_or_rur']
    c2008 = row['2008urb_or_rur']
    
    if c2008 == c2018: # Same category for both years
        wa_blocks.loc[index, 'block_change'] = 'No Change in Category'
    elif c2008 == 'Urban' and c2018 == 'Rural': # Changed from Urban to Rural
        wa_blocks.loc[index, 'block_change'] = 'De-Urbanized'
    elif c2008 == 'Rural' and c2018 == 'Urban': # Changed from Rural to Urban
        wa_blocks.loc[index, 'block_change'] = 'Urbanized'

In [48]:
# Step 5

# Variables for totals
urb_grps = 0
deurb_grps = 0
nochange = 0

for index, row in wa_blocks.iterrows():
    if row['block_change'] == 'De-Urbanized':
        deurb_grps = deurb_grps + 1
    elif row['block_change'] == 'Urbanized':
        urb_grps = urb_grps + 1
    else: # no change
        nochange = nochange + 1
        
print str(urb_grps) + " block groups 'urbanized' between 2008 and 2018, while " + str(deurb_grps) + " groups de-urbanized in the same time period. \n" + str(nochange) + " block groups had no change in category."

65 block groups 'urbanized' between 2008 and 2018, while 2 groups de-urbanized in the same time period. 
4716 block groups had no change in category.


In [18]:
# Delete later - printing which blocks de-urbanized

for index, row in wa_blocks.iterrows():
    if row['block_change'] == 'Urbanized':
        print wa_blocks.loc[index, ['density', 'urb_or_rur', '2008density', '2008urb_or_rur','block_change']]
        print '\n'

density                947.903
urb_or_rur               Rural
2008density            1047.24
2008urb_or_rur           Urban
block_change      De-Urbanized
Name: 2272, dtype: object


density                986.513
urb_or_rur               Rural
2008density            1005.36
2008urb_or_rur           Urban
block_change      De-Urbanized
Name: 2741, dtype: object




In [26]:
# Delete later - shows the different categories and their respective counts

wa_blocks['block_change'].value_counts()

No Change in Category    4716
Urbanized                  65
De-Urbanized                2
Name: block_change, dtype: int64

In [49]:
# This saves the result as a shapefile

#wa_blocks.to_file("wa_blocks.shp")